# Children Act Applications

### This code includes the following processes:
1. Children Act applications data extraction, creating of child level dataset
2. Creation of order level dataset
3. Creation of application (events) level dataset
4. Creation of case starts daatset
5. Individual children (annual and quarterly)
6. Extaction of High court information and creation of high court flag
7. Extration of parties data and counting applicants/respondents in a case

#### Import packages and set options

In [ ]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

#### Defining variables

In [ ]:
# define some variables to be used throughout the notebook
db1 = "familyman_dev_v2" #database where Familyman data is stored
snapshot_date = "2021-11-11" #To update where necessary
db2 = "fcsq" #database where tables created as part of FCSQ processing are stored where required

In [ ]:
# create path for within athena FCSQ database in the S3 folder
fcsq_db_path = f"s3://alpha-family-data/fcsq_processing/CA_apps/"

## 1. Applications and children

#### 1.1. Applications - table of all applications, filtered by specific order types

In [ ]:
# Creating a table with all the applications for CA, codes selected from the order type lookup file.
# The cross join unnest function flattens the application values to one row per app type
pydb.create_temp_table( 
f"""
SELECT 
    e.case_number,
    e.receipt_date,
    EXTRACT(year FROM e.receipt_date) AS year,
    EXTRACT(quarter FROM e.receipt_date) AS quarter,
    f.event,
    f.field_model,
    f.value as all_event_app_types,
    TRIM(ord_type) as order_type,
    CAST(SUBSTR(CAST(f.event AS varchar),1,3) AS integer) AS court_code
  FROM 
    {db1}.event_fields F
    INNER JOIN {db1}.events e
      ON f.event = e.event
    CROSS JOIN UNNEST(SPLIT(f.value,',')) AS t(ord_type)
  WHERE 
    field_model IN('U22_AT','G50_AT')
    AND   TRIM(ord_type) IN (SELECT 
                               order_type 
                             FROM 
                              {db2}.order_type_lookup
                             WHERE 
                               child_act = 'Y')
    AND e.error = 'N'
    AND f.mojap_snapshot_date = DATE'{snapshot_date}'
    AND e.mojap_snapshot_date = DATE'{snapshot_date}'
""",

"ca_apps")

#### 1.2. Children (events) - joining children onto the application event where children are recorded on the event

In [ ]:
# Only events with children recorded are included. Children not recorded in the value field are dealt with in the following step
# As with the previous code above, the cross join unnest flattens the child data to 1 row per child recorded against the application (event)
pydb.create_temp_table( 
f"""
  SELECT 
    a.*,
    f.value children,
    TRY_CAST(TRIM(child_role_id) as bigint) child_role_id
  FROM 
    __temp__.ca_apps a
    LEFT JOIN {db1}.event_fields f
      ON f.event = a.event
   CROSS JOIN UNNEST(SPLIT(f.value,',')) AS t(child_role_id)
  WHERE f.field_model IN('U22_CH','G50_CH')
    AND child_role_id <> ''
    AND f.mojap_snapshot_date = DATE'{snapshot_date}'
""",

"ca_apps_child_event")

#### 1.2.1 Adding child characteristics for children recoreded on the event

In [ ]:
#Linking to the party table to get date of birth and gender of the child. 
#Removes children that are flagged to delete in the roles table - these orders will be added to the child not recorded table and dob and gender will be unknown*/
pydb.create_temp_table( 
f"""
  SELECT 
    a.case_number,
    a.receipt_date,
    a.year,
    a.quarter,
    a.event,
    a.field_model,
    a.court_code,
    a.order_type,
    a.child_role_id,
    p.dob,
    p.gender
  FROM
    __temp__.ca_apps_child_event a
    JOIN {db1}.roles r ON a.child_role_id = r.role
    JOIN {db1}.parties p ON r.party = p.party 
  WHERE
    r.delete_flag = 'N'
    AND r.mojap_snapshot_date = DATE'{snapshot_date}'
    AND p.mojap_snapshot_date = DATE'{snapshot_date}'
 """,

 "ca_apps_child_event_2")  

#### 1.3. Children (case) - joining the children onto the application data using the roles table where children are not recorded on the event. 

In [ ]:
# Take all events where no children were recorded against the event in the children(events) table and get children details from the roles/parties tables.
# Where there are no children recorded on the event then all children recorded in the case are considered involved in the application - this is not an assumption but based on how Familyman data entry works 
# This table does not include those where children are recorded on the event and are flagged to delete in the roles table. These children are included in the next stage
# Adding date of birth and gender from the parties table
pydb.create_temp_table( 
f"""
  SELECT
    a.*,
    r.role child_role_id,
    p.dob,
    p.gender
  FROM
    __temp__.ca_apps a
    JOIN {db1}.roles r 
      on a.case_number = r.case_number
    JOIN {db1}.parties p 
      on r.party = p.party 
  WHERE
    event NOT IN (SELECT event FROM __temp__.ca_apps_child_event)
    AND role_model in ('CHLDC', 'CHLDZ')
    AND r.delete_flag = 'N'
    AND r.mojap_snapshot_date = DATE'{snapshot_date}'
    AND p.mojap_snapshot_date = DATE'{snapshot_date}'
""",

"ca_apps_child_case")

#### 1.4. Children (not recorded) - children that are not recorded on the event or the case. 

In [ ]:
#There are a small number of applicaitons where the children are not reocorded on the event and there are also no children recorded on the case (or all chilren recorded have a delete flag of 'Y'). This tends to be for DV cases, where CA order types 
#   may have been made but children are not necessarily recorded on the case. Here we default to just one child per order/disposal, with unknown characteristics. This is an assumption and will not always be correct.
pydb.create_temp_table(
f"""
  SELECT
    *,
    CAST(NULL AS INTEGER) AS child_role_id,
    CAST(NULL AS DATE) dob,
    CAST(NULL AS INTEGER) AS gender
  FROM
    __temp__.ca_apps
  WHERE
    event NOT IN (SELECT event FROM __temp__.ca_apps_child_event_2)
    AND event NOT IN (SELECT event FROM __temp__.ca_apps_child_case)
""",

"ca_apps_no_child")

#### 1.5. Appending children from events, children from case and no child data sets

In [ ]:
# Unioning all the children data sets together
pydb.create_temp_table( 
f"""
  SELECT 
    case_number,
    receipt_date,
    year,
    quarter,
    event,
    field_model,
    court_code,
    order_type,
    child_role_id,
    dob,
    gender
  FROM
    __temp__.ca_apps_child_event_2
  UNION ALL
  SELECT
    case_number,
    receipt_date,
    year,
    quarter,
    event,
    field_model,
    court_code,
    order_type,
    child_role_id,
    dob,
    gender
  FROM
    __temp__.ca_apps_child_case
  UNION ALL
  SELECT
    case_number,
    receipt_date,
    year,
    quarter,
    event,
    field_model,
    court_code,
    order_type,
    child_role_id,
    dob,
    gender
  FROM
    __temp__.ca_apps_no_child
""",

 "ca_apps_all_children")

#### Adding the order type details

In [ ]:
# Order type code and description are added from the lookup. Lookup has been created independently from this process.
pydb.create_temp_table( 
f"""
SELECT
    a.*,
    l.order_code,
    l.order_desc
FROM 
   __temp__.ca_apps_all_children a
   LEFT JOIN {db2}.order_type_lookup l
     ON a.order_type = l.order_type
""",

"ca_apps_order_type")

#### 1.6 Ranking duplicate child/order type data within a case

In [ ]:
# The apps all children table contains some records with the same order type for the same child in the same case in the same year. 
# Here we order the cases so that in those cases initial apps are ranked earlier than subsequent apps, and where the ord type is the same the earliest app is ranked highest
pydb.create_temp_table( 
f"""
SELECT
    *,
    Floor((date_diff('day',cast(dob as date),cast(receipt_date as date)))/365.25) AS child_age,
    substr(case_number, 5,1) as case_type,
    ROW_NUMBER() OVER(PARTITION BY year, case_number, child_role_id, order_desc
                       ORDER BY case_number, child_role_id, order_desc, field_model DESC, receipt_date ASC) 
      AS dup_rank
FROM 
   __temp__.ca_apps_order_type
""",

"ca_apps_dup_rank")

## 1.7 Children involved in applications dataset 
#### Children counted for every order type applied for

In [ ]:
#drop table in Athena (if it already exists)
_ = pydb.start_query_execution_and_wait(f"""DROP TABLE {db2}.ca_apps_child""")

In [ ]:
# Select the ealliest record for each duplicate order type per child
# Add whether public or private law case type (potentially revisit this as private law may contain adoption cases)

#set S3 file path
app_child_s3_path = os.path.join(fcsq_db_path, "app_child/")
# Delete all the underlying data stored within the S3 location
if wr.s3.list_objects(app_child_s3_path):
    print("deleting objs")
    wr.s3.delete_objects(app_child_s3_path)
    
#Create table in Athena

t_child =  f"""
CREATE TABLE {db2}.ca_apps_child WITH
(
    external_location='{app_child_s3_path}'
) AS
SELECT
    year,
    quarter,
    case_number,
    receipt_date,
    event,
    field_model,
    order_type,
    order_code,
    order_desc,
    child_role_id,
    CASE WHEN gender = 1 THEN 'Male'
         WHEN gender = 2 THEN 'Female'
        ELSE 'Unknown'
      end as Gender,
    dob,
    child_age,
    CASE WHEN Child_age is Null
          THEN 'Unknown'
         WHEN Child_age < 0
          THEN 'Other'
         WHEN Child_age < 1
          THEN '<1 year'
         WHEN Child_age<5
           THEN '1-4 years'
         WHEN Child_age<10
          THEN '5-9 years'
         WHEN Child_age<15
          THEN '10-14 years'
         WHEN Child_age<18
          THEN '15-17 years'
           ELSE 'Other'
      END AS Age_band,
    court_code,
    CASE WHEN order_type in ('CRO','SSC','DCO','OSA','SO','DSO','OC','OCST','ARC','ARST','ESO','XESO','CAO','EPO','XEPO','DEPO','WEP')
      OR case_type = 'C' THEN 'C' Else 'P' END 
     AS order_case_type
FROM 
   __temp__.ca_apps_dup_rank
WHERE
  dup_rank = 1
"""

_ = pydb.start_query_execution_and_wait(t_child)

## 2. Orders applied for

In [ ]:
#drop table in Athena (if it already exists)
_ = pydb.start_query_execution_and_wait(f"""DROP TABLE {db2}.ca_apps_order_types""")

In [ ]:
# Remove the child ID and group up so we only count an order type within an event once, rather than per child

#set S3 file path
app_orders_s3_path = os.path.join(fcsq_db_path, "app_order_types/")
# Delete all the underlying data stored within the S3 location
if wr.s3.list_objects(app_orders_s3_path):
    print("deleting objs")
    wr.s3.delete_objects(app_orders_s3_path)

t_orders = f"""
CREATE TABLE {db2}.ca_apps_order_types WITH
(
    external_location='{app_orders_s3_path}'
) AS
SELECT 
  DISTINCT 
    year,
    quarter,
    case_number,
    order_case_type,
    receipt_date,
    event,
    court_code,
    order_type,
    order_code,
    order_desc
FROM 
  {db2}.ca_apps_child
"""

_ = pydb.start_query_execution_and_wait(t_orders)

## 3. Application count data

In [ ]:
# Counting applications (individual events). Here multiple orders applied for under one event are only counted once
pydb.create_temp_table( 
f"""
SELECT 
  DISTINCT 
    year,
    quarter,
    case_number,
    order_case_type,
    CASE WHEN SUBSTR(case_number,5,1) NOT IN ('P','C')
           THEN 'NA'
         WHEN order_case_type = 'C' 
              AND case_number in (SELECT case_number
                                  FROM {db2}.ca_apps_order_types
                                  WHERE order_case_type = 'P')
           THEN 'P'
        ELSE order_case_type END
      AS main_case_type,
    receipt_date,
    event,
    court_code
FROM 
  {db2}.ca_apps_child
""",

"ca_apps_event_count")

## 4. Case count data

In [ ]:
# Keeping just one record per case, and selecting the earliest record
pydb.create_temp_table( 
f"""
SELECT    
  main_case_type,    
  case_number,
  (MIN(receipt_date)) AS MIN_of_RECEIPT_DATE,
  EXTRACT (YEAR FROM (MIN(receipt_date))) AS Year,
  EXTRACT (QUARTER FROM (MIN(receipt_date))) AS Quarter
FROM 
  __temp__.ca_apps_event_count
GROUP BY 
  main_case_type, 
  case_number
""",

 "ca_apps_case_count")

## 5. Individual children

#### 5.1 Individual children by year

In [ ]:
# Adding a count for the number of times an individual child appears within a year, and then selecting the earliest record
pydb.create_temp_table( 
f"""
WITH ca_apps_count_child_yr AS (

SELECT    
  *,    
  ROW_NUMBER() OVER(PARTITION BY year, child_role_id, order_case_type
                       ORDER BY child_role_id, year, receipt_date ASC) 
      AS child_count_yr
FROM 
  {db2}.ca_apps_child
  
)
  
SELECT    
  year, 
  quarter,
  case_number,
  child_role_id,
  gender,
  dob,
  age_band,
  order_case_type
FROM 
  ca_apps_count_child_yr
WHERE
  child_count_yr = 1 
""",

"ca_apps_ind_child_yr")

#### 5.2 Individual children by quarter

In [ ]:
# Adding a count for the number of times an individual child appears within a quarter, and then selecting the earliest record
pydb.create_temp_table( 
f"""
WITH ca_apps_count_child_qtr AS (

SELECT    
  *,    
  ROW_NUMBER() OVER(PARTITION BY year, quarter, child_role_id, order_case_type
                       ORDER BY child_role_id, year, quarter, receipt_date ASC) 
      AS child_count_qtr
FROM 
  {db2}.ca_apps_child

)

SELECT    
  year, 
  quarter,
  case_number,
  child_role_id,
  gender,
  dob,
  age_band,
  order_case_type
FROM 
  ca_apps_count_child_qtr
WHERE
  child_count_qtr = 1
  
""",

"ca_apps_ind_child_qtr")

## 6. High Court

#### 6.1 Extracting High Court cases

In [ ]:
# extracting high court cases listed against the case field table.
# there is also a HC event field model (U22_HC). Howver, HMCTS advice that we use the case field as this could be updated during the case (and the event field can't), and so won't be as accurate
# There are some duplicate case numbers with different receipt days so we rank and take the earliest
# the creating court is taken from the court listed against the U22 (proceedings issued) event
pydb.create_temp_table( 
f"""
WITH high_court_cases AS (

SELECT
  DISTINCT
  c.case_number,
  c.value as case_HC_value,
  e.creating_court,
  CASE WHEN e.creating_court in ('EC','FD','IL','LB','WT','ZC')
        THEN 'Central London DFJ'
          ELSE 'Not Central London DFJ'
    END AS HC_London_Ind,
  ROW_NUMBER() OVER(PARTITION BY c.case_number, c.value
                       ORDER BY c.case_number, c.value DESC, receipt_date ASC) 
      AS case_rank
FROM
  {db1}.case_fields c
  LEFT JOIN {db1}.events e
    on c.case_number = e.case_number
WHERE
  (c.field_model = 'FM2C_HC' AND c.value = 'Y') 
  AND e.event_model = 'U22'
  AND e.error = 'N'
  AND c.mojap_snapshot_date = DATE'{snapshot_date}'
  AND e.mojap_snapshot_date = DATE'{snapshot_date}'
  
)

SELECT
  *
FROM 
  high_court_cases
WHERE 
  case_rank = 1;

""",

"ca_high_court_cases")

#### 6.2 Linking High Court cases the the cases starts table

In [ ]:
# link high court cases to original case table
pydb.create_temp_table( 
f"""
SELECT
  c.*,
  h.case_HC_value,
  h.creating_court AS HC_U22_court,
  h.HC_London_ind
FROM
  __temp__.ca_apps_case_count c
  LEFT JOIN __temp__.ca_high_court_cases h
    ON c.case_number = h.case_number
""",
    
"ca_cases_HC")

## 7.Parties

#### 7.1 Creating a table with applicants and respondents

In [ ]:
# pick out applicant and respondents from the roles table along with gender details from the party table.
# currently applicants and respondents from adoption cases are included as there are some adoption cases included within private law but this will be revisited
pydb.create_temp_table( 
f"""
  SELECT
    DISTINCT
    r.role_model,
    CASE WHEN r.role_model in ('APLC','APLZ','APLA')
          THEN 'Applicant'
         WHEN r.role_model in ('RSPC','RSPZ','RSPA')
          THEN 'Respondent'
      END AS case_role, 
    r.role role_id,
    r.party as party_id,
    r.case_number,
    p.gender
  FROM
    {db1}.roles r 
    JOIN {db1}.parties p 
      on r.party = p.party 
  WHERE
    r.role_model in ('APLC','APLZ','APLA','RSPC','RSPZ','RSPA')
    AND r.mojap_snapshot_date = DATE'{snapshot_date}'
    AND p.mojap_snapshot_date = DATE'{snapshot_date}'
    AND r.delete_flag = 'N'
""",

"app_resp")

#### 7.2 Linking applicants to case starts

In [ ]:
# When linking we also add a count to how many applicants are recorded on the case
# As this is being counted against case starts we are not including applicants in subsequent applications (G50 events)
pydb.create_temp_table(
f"""
  SELECT
    c.year,
    c.quarter,
    c.MIN_of_RECEIPT_DATE,
    c.case_number,
    c.main_case_type,
    r.role_model,
    r.case_role, 
    r.role_id,
    r.party_id,
    r.gender,
    ROW_NUMBER() OVER(PARTITION BY c.case_number, main_case_type
                      ORDER BY c.case_number, main_case_type) 
      AS applicant_count
  FROM
    __temp__.ca_apps_case_count c 
    JOIN __temp__.app_resp r 
      on c.case_number = r.case_number 
  WHERE
    r.case_role = 'Applicant'
""",

"applicants")

#### 7.3 Linking respondents to case starts

In [ ]:
# When linking we also add a count to how many respondents are recorded on the case
# As this is being counted against case starts we are not including respondents in subsequent applications (G50 events)
pydb.create_temp_table(
f"""
  SELECT
    c.year,
    c.quarter,
    c.MIN_of_RECEIPT_DATE,
    c.case_number,
    c.main_case_type,
    r.role_model,
    r.case_role, 
    r.role_id,
    r.party_id,
    r.gender,
    ROW_NUMBER() OVER(PARTITION BY c.case_number, main_case_type
                      ORDER BY c.case_number, main_case_type) 
      AS respondent_count
  FROM
    __temp__.ca_apps_case_count c 
    JOIN __temp__.app_resp r 
      on c.case_number = r.case_number 
  WHERE
    r.case_role = 'Respondent'
""",

"respondents")

#### 7.4 Counting applicants and respondents in each case

In [ ]:
#Linking the count of applicants in a case to the count of respondents in a case
pydb.create_temp_table( 
f"""
  SELECT
    c.year,
    c.quarter,
    c.case_number,
    c.main_case_type,
    MAX(a.applicant_count) as No_of_applicants,
    MAX(r.respondent_count) as No_of_respondents
  FROM
    __temp__.ca_apps_case_count c 
    JOIN __temp__.applicants a 
      on c.case_number = a.case_number
    JOIN __temp__.respondents r
      on c.case_number = r.case_number  
  GROUP BY
    c.year,
    c.quarter,
    c.main_case_type,
    c.case_number
""",
    
"party_count")
